## 瀏覽器&套件設定

In [31]:
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

In [32]:
# 擋掉通知黑屏，以防止瀏覽器第一次登入時跳出通知視窗
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.notifications": 2
})

## 自動登入

In [33]:
# ------ 登入的帳號與密碼 ------
username = str(input("請輸入帳號："))
password = str(input("請輸入密碼："))

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

url = 'https://www.facebook.com'  
driver.get(url)

WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//*[@id="email"]')))

elem = driver.find_element(By.ID,"email")
elem.send_keys(username)

elem = driver.find_element(By.ID,"pass")
elem.send_keys(password)


# 提交登入
elem.send_keys(Keys.RETURN)
time.sleep(5)

# 檢查有沒有被擋下來
if len(driver.find_elements(By.XPATH,"//*[contains(text(), '你的帳號暫時被鎖住')]")) > 0:
    driver.find_elements(By.XPATH,"//*[contains(text(), '是')]")[1].click()
elif len(driver.find_elements(By.XPATH, "//*[contains(text(), '用戶名稱或密碼無效')]")) > 0:
    print("登入失敗，請檢查帳號密碼是否正確")
elif len(driver.find_elements(By.XPATH, "//*[contains(text(), 'Invalid username or password')]")) > 0:
    print("登入失敗，請檢查帳號密碼是否正確")
else:
    feed = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="main"]'))) # 等待動態頁面(鎖定貼文)加載完成
    print("登入成功")

登入成功


## 純貼文爬取

In [34]:
posts_number = int(input("請輸入要爬取的貼文數量:"))
posts = [] 


# 初始設定
last_height = driver.execute_script("return document.body.scrollHeight") # 取得頁面高度初始值
last_count = 0 # 計算目前貼文數量

feed = driver.find_element(By.CSS_SELECTOR, 'div[role="main"]') # 找到動態頁面(鎖定貼文)
while True:

    # 如果已經找到足夠的貼文，則停止
    if len(posts) >= posts_number:
        posts = posts[:posts_number]
        break
    
    # 貼文文字的 class selector
    post_text_class_selector = 'div[data-ad-preview="message"]'
    
    # 捕捉所有貼文元素
    
    elements = feed.find_elements(By.CSS_SELECTOR, 'div[class = x1lliihq]')
    current_count = len(elements)    
    if current_count == 0:
        elements = feed.find_elements(By.CSS_SELECTOR, '[data-pagelet]')
        current_count = len(elements)
    print(last_count,current_count)
    

    if current_count != last_count:
        try:
            print("!!! 檢查當前是否有 「查看更多」 Button !!!")
            buttons = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(text(), '查看更多') and @role='button']")))
            for button in buttons:

                # 滾動到按鈕位置
                driver.execute_script("arguments[0].scrollIntoView(true);", button)
                WebDriverWait(driver, 5).until(EC.visibility_of(button))
                # 使用 JavaScript 強制點擊按鈕
                driver.execute_script("arguments[0].click();", button)
                print('Button clicked!')
        except Exception as e:
            print(f"當前無「查看更多」 Button")
            pass

        print('!!! 檢查完畢 !!!')
        # 只處理新出現的 elements
        new_elements = elements[last_count:]
        for element in new_elements:
            try:
                print('--------------貼文分隔線--------------')
                element = element.find_element(By.CSS_SELECTOR, post_text_class_selector)
                posts.append(element.text)
                print(element.text)
            except:
                print('此篇非普通貼文，跳過處理')
        print('---------------------------------')
        # 更新 last_count
        last_count = current_count

    # 計算新的滾動高度並比較是否到達底部
    new_height = driver.execute_script("return document.body.scrollHeight")
    last_height = new_height

    # 滾動到頁面底部
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 等待新內容加載(秒)
    time.sleep(4)

print('共爬取', len(posts), '篇貼文')
print(posts)
driver.refresh()

0 2
!!! 檢查當前是否有 「查看更多」 Button !!!
當前無「查看更多」 Button
!!! 檢查完畢 !!!
--------------貼文分隔線--------------
此篇非普通貼文，跳過處理
--------------貼文分隔線--------------
7-11店到店貨到付款，中和中山路二段228號面交
---------------------------------
2 8
!!! 檢查當前是否有 「查看更多」 Button !!!
Button clicked!
Button clicked!
!!! 檢查完畢 !!!
--------------貼文分隔線--------------
此篇非普通貼文，跳過處理
--------------貼文分隔線--------------
此篇非普通貼文，跳過處理
--------------貼文分隔線--------------
緊急
緊急
該電話帳戶不屬於2000FUN
如受此電話帳戶行騙 請即與警方聯絡
2000FUN客服唯一電話帳戶
 客服電話： 2110 2210
WhatsApp 電話：6433 5478
2000Fun砌機部 WhatsApp 電話： 6368 2417
--------------貼文分隔線--------------
準備好在 MSI 獲得免費贈禮吧！ 
分享您的 MSI 產品，就有機會獲得價值 30 美元的 Steam Code！
立即探索 
 https://msi.gm/S32E614A
#ShoutOutForMSI #MSI #MSIQDOLED #電競 #主機 #桌機 #螢幕 #顯示器
--------------貼文分隔線--------------
收你不要的電腦主機/筆電/電腦零件
--------------貼文分隔線--------------
舊手掣最多可當
換新手掣
Xbox推出手掣舊換新計劃
準備更好裝備迎接《艾爾登法環 黃金樹幽影》、《Senua’s Saga: Hellblade II》等勁作啦！
三款主機品牌手掣都可以Trade架！*
以舊手掣換購Xbox無線手掣可享折扣優惠*：
1, 換購Xbox Elite無線手掣Series 2 (任何顏色)
可獲
折扣
2, 換購Xbox無線手掣 (任何顏色|不包括Xbox無線